#  Intel® AI Reference Models Jupyter Notebook

This Jupyter notebook helps you choose and run a comparison between two models from the Intel® AI Reference Models repo using Intel® Optimizations for TensorFlow*, Intel® extension for TensorFlow, Intel® Optimizations for PyTorch and Intel® extension for PyTorch. When you run the notebook, it installs required package dependencies, displays information about your platform, lets you choose the two models to compare, runs those models, and finally displays a performance comparison chart.

# Workflow of the notebook

In [ ]:
!pip install graphviz

In [ ]:
from graphviz import Digraph
from IPython.display import Image, display

# Create a Digraph object
general_graph = Digraph()

# Define the nodes
general_graph.node('G', 'General Specification')

general_graph.node('F', 'Framework')
general_graph.node('PyT', 'PyTorch')
general_graph.node('TF', 'TensorFlow')

general_graph.node('HW', 'Hardware')
general_graph.node('CPU', 'CPU')
general_graph.node('GPU', 'GPU')

general_graph.node('Flex', 'Flex')
general_graph.node('Max', 'Max')
general_graph.node('Arc', 'Arc')


# Define the edges
general_graph.edges([('G','F'), ('F','PyT'), ('F','TF'), ('PyT', 'HW'), ('TF', 'HW'), ('HW', 'CPU'), ('HW', 'GPU'), ('GPU', 'Flex'), ('GPU', 'Max'), ('GPU', 'Arc')])
# Save the graph to a file
general_graph.render('simple_flow_chart', format='png', cleanup=True)

# Display the image in the notebook
display(Image(filename='simple_flow_chart.png'))

model_graph = Digraph()
model_graph.node('M','Model Specification')
model_graph.node('TM','Test_Mode')
model_graph.node('Mo','Mode')
model_graph.node('P','Precision')
model_graph.node('S','Script')
model_graph.node('AS','Additional Setup')
model_graph.node('PL','Pretrained/Log Directory')
model_graph.node('R','Run')

model_graph.edges([('M','TM'), ('M','Mo'),('M','P'),('M','S'),('TM','AS'),('Mo','AS'),('P','AS'),('S','AS'),('AS','PL'),('PL','R')])
model_graph.render('simple_flow_chart_1', format='png', cleanup=True)
display(Image(filename='simple_flow_chart_1.png'))

<a id='section_1'></a>
# Step 1: Display Platform Information 

Install the required dependencies for the jupyter notebook

In [ ]:

!pip install matplotlib ipykernel psutil pandas cxxfilt gitpython wheel pyyaml ipython
!pip install gcg
!python3 -m pip install gitpython
!pip install prettytable
!pip install --upgrade matplotlib
!pip install ipywidgets

Select a framework from the dropdown option below the next cell and install the required framework packages

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from profiling.profile_utils import GeneralConfigFile, AIReferenceConfigFile
from prettytable import PrettyTable
import subprocess
import os

# Set environment variables
os.environ['no_proxy'] = ""
os.environ['NO_PROXY'] = ""
general_config_1 = GeneralConfigFile(ai_root=os.getcwd()+'/..')
general_config_1.toggle_events()

In [ ]:
if (general_config_1.hardware_dropdown.value and general_config_1.framework_dropdown.value and (general_config_1.hardware_dropdown.value == 'CPU' or (general_config_1.hardware_dropdown.value == 'GPU' and general_config_1.gpu_series_dropdown.value))):
    current_dir = current_directory = os.getcwd()
    selected_hardware = general_config_1.hardware_dropdown.value
    selected_framework = general_config_1.framework_dropdown.value

    if selected_framework == 'PyTorch':
        if selected_hardware == 'CPU':
            !python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
            !python -m pip install intel-extension-for-pytorch
            !python -m pip install oneccl_bind_pt --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/

            print("PyTorch CPU packages installed.")

            !./profiling/pytorch_cpu_setup.sh

            !pip install packaging intel-openmp

            os.environ["LD_PRELOAD"] = f"{current_dir}/jemalloc/lib/libjemalloc.so:{current_dir}/tcmalloc/lib/libtcmalloc.so:{general_config_1.root}/new_env/lib/libiomp5.so"
            os.environ["MALLOC_CONF"] = "oversize_threshold:1,background_thread:true,metadata_thp:auto,dirty_decay_ms:9000000000,muzzy_decay_ms:9000000000"
            os.environ["DNNL_MAX_CPU_ISA"] = "AVX512_CORE_AMX"
            os.environ['yaml_file'] = 'profiling/ai_reference_models_pytorch_cpu.yaml'

            print("Finish building PyTorch for CPU")

        elif selected_hardware == 'GPU':
            install_oneapi = general_config_1.intel_oneapi_dropdown.value
            gpu_series = general_config_1.gpu_series_dropdown.value

            if (install_oneapi):
                print("Intel® oneAPI Base Toolkit is installed")
                print("Installing PyTorch for GPU...")
                !pip install torch==2.1.0a0 torchvision==0.16.a0 torchaudio==2.1.0.a0 intel-extension-for-pytorch==2.1.10+xpu --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/xpu/us/
                print("IPEX GPU packages installed.")

                if gpu_series == 'Flex':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_flex_gpu.yaml"
                elif gpu_series == 'Max':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_max_gpu.yaml"
                elif gpu_series == 'Arc':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_arc_gpu.yaml"

                oneapi_path = general_config_1.oneapi_path_input.value.strip()
                print("Finish building IPEX for GPU")
                os.environ['oneAPI_PATH'] = oneapi_path
                oneAPI_PATH = os.environ.get('oneAPI_PATH', '')
                script_content = f"""
                source /{oneAPI_PATH}/compiler/latest/env/vars.sh
                source /{oneAPI_PATH}/mkl/latest/env/vars.sh
                source /{oneAPI_PATH}/tbb/latest/env/vars.sh
                source /{oneAPI_PATH}/mpi/latest/env/vars.sh
                source /{oneAPI_PATH}/ccl/latest/env/vars.sh
                """
                subprocess.run(["/bin/bash", "-c", script_content], check=True)
                print("One API sourcing done.")
                print("PyTorch GPU packages installed.")

    elif selected_framework == 'TensorFlow':
        if selected_hardware == 'CPU':
            !pip cache purge
            !pip install --upgrade tf-nightly-cpu
            !pip install --upgrade tf-keras-nightly
            !pip install --upgrade tf-estimator-nightly
            print("TensorFlow packages installed.")
            os.environ['yaml_file'] = 'profiling/ai_reference_models_tensorflow_cpu.yaml'
            print("Finish building TF for CPU")

        elif selected_hardware == 'GPU':
            gpu_series = general_config_1.gpu_series_dropdown.value
            print("Installing TensorFlow and ITEX for GPU...")
            !pip install tensorflow==2.15.0

            !pip install --upgrade intel-extension-for-tensorflow[xpu]

            if gpu_series == 'Flex':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_flex_gpu.yaml"
            elif gpu_series == 'Max':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_max_gpu.yaml"
            elif gpu_series == 'Arc':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_arc_gpu.yaml"
            print("TensorFlow GPU packages installed.")

else:
    raise Exception("Please rerun the selections to ensure correct installations.")


Set the path for the AI reference models, assumed to be the current working directory, and the path for the utility functions directory inside the cloned Intel® AI Reference Models directory.

In [ ]:
# If default path does not work, change AIReferenceRoot path according to your environment
os.environ['NotebookRoot'] = os.getcwd()
os.environ['AIReferenceRoot'] = os.environ['NotebookRoot'] + os.sep
os.environ['ProfileUtilsRoot'] = os.environ['AIReferenceRoot'] + os.sep + "notebooks/profiling/"
print("Path for the notebook root is: ", os.environ['NotebookRoot'])
print("Path for the AI Reference Models root is: ", os.environ['AIReferenceRoot'])
print("Path for utility functions directory is: ", os.environ['ProfileUtilsRoot'])

# Check for mandatory python scripts after AIReferenceRoot and ProfileUtilsRoot are assigned
import os
current_path = os.getcwd()
benchmark_path = os.environ['AIReferenceRoot'] + os.sep + "benchmarks/launch_benchmark.py"
if os.path.exists(benchmark_path) == True:
    print(benchmark_path)
else:
    print(benchmark_path)
    print("ERROR! Can't find benchmark/launch_benchmark.py script!")

profile_utils_path = os.environ['ProfileUtilsRoot'] + "profile_utils.py"
if os.path.exists(profile_utils_path) == True:
    print(profile_utils_path)
else:
    print("ERROR! Can't find profile_utils.py script!")

## Step 2: Run the first model

This notebook helps you compare the performance of two models listed in the supported models. Select the first model to run and compare.  (If the environment variable MODEL_1_INDEX is set, we'll use that instead of prompting for input.)

### Step 2.1: Display all the supported models and select first  model to run.

In [ ]:
config_1 = AIReferenceConfigFile(os.environ['yaml_file'], os.getcwd() + '/..')
notebook_root = config_1.notebook_root
%cd $notebook_root
selected_model = config_1.model_selection()
print(selected_model)
selected_model = config_1.device_specific(selected_model, general_config_1.hardware_dropdown.value, os.environ['yaml_file'], selected_framework)
os.environ['PRECISION'] = config_1.precision
if selected_framework == 'TensorFlow' and selected_hardware == 'CPU':
    selected_script = config_1.script
else:
    os.environ['TEST_MODE'] = config_1.test_mode

## Additional Setup

The following cell checks for additional environment variables that needs to be specified per model specification.

Then checks for additional files, libraries, and configurations that needs to be installed per model specification. Commands are installed in a chronological dependency order.

In [ ]:
import subprocess
ai_reference_root = config_1.ai_root
%cd $ai_reference_root

for export in selected_model['exports']:
    env_name = export.split(' ')[0]
    env_value = export.split(' ')[1]
    if env_value == '-i':
        env_value = input(f'Please provide the path for the following environment variable {env_name}: ', )
    os.environ[env_name] = os.path.expandvars(env_value)

    print("Assigned environment variable ", env_name, " : ", os.environ[env_name])

for cmd in selected_model['additional-commands']:
    if 'os.environ' in cmd: exec(cmd)
    else : subprocess.call(cmd, shell=True)

In [ ]:
import os
import subprocess

# Retrieve the AI reference root from environment variables
ai_reference_root = os.environ.get('AIReferenceRoot')
if not ai_reference_root:
    raise ValueError("Environment variable 'AIReferenceRoot' is not set.")

# Change to the specified directory
os.chdir(ai_reference_root)

# Process exports from the selected model
exports = selected_model.get('exports', [])
for export in exports:
    try:
        # Split the export string into name and value
        env_parts = export.split(' ', 1)
        if len(env_parts) != 2:
            raise ValueError(f"Invalid export format: {export}")

        env_name, env_value = env_parts
        if env_value == '-i':
            # Prompt for user input if value is '-i'
            env_value = input(f"Please provide the path for the environment variable '{env_name}': ")

        # Expand and set the environment variable
        os.environ[env_name] = os.path.expandvars(env_value)
        print(f"Assigned environment variable {env_name}: {os.environ[env_name]}")

    except Exception as e:
        print(f"Error processing export '{export}': {e}")

# Process additional commands from the selected model
additional_commands = selected_model.get('additional-commands', [])
for cmd in additional_commands:
    try:
        if 'os.environ' in cmd:
            # Execute Python commands directly
            exec(cmd)
        else:
            # Execute shell commands
            subprocess.call(cmd, shell=True)
        print(f"Executed command: {cmd}")

    except Exception as e:
        print(f"Error executing command '{cmd}': {e}")


### Step 2.2: Get the required dataset for the selected model

The following cell checks the dataset path. If the data-location is already specified, then the notebook will use the dataset path mentioned in ai_reference_models.ini file. If the data-location is not specified in the ai_reference_models.ini, the data downloading option instructions are shown. You must manually download the dataset using these instructions before you can proceed to the next cell.

Set the path for dataset directory for the first model:

DATASET_DIR': the path where the dataset exists and is downloaded.

**ACTION : You need to input the path where the dataset for the first model exists or where you have downloaded it in your system**

In [ ]:
dataset_path = input('Input the path where the dataset exists for the first model:')

os.environ['DATASET_DIR'] = dataset_path

print("Data location path:", os.environ['DATASET_DIR'])

### Step 2.3: Prepare pre-trained model for the selected model

This step checks if the pre-trained model for the selected model exists in the pre-trained directory path. If the pre-trained directory does not exist, then it downloads the pre-trained model for the selected precision.

In [ ]:
wget_value = config_1.wget

pretrain_model_path = config_1.download_pretrained_model(wget=wget_value, current_path=config_1.notebook_root)
pretrain_model_path = config_1.uncompress_file(pretrain_model_path, current_path=config_1.notebook_root)
if pretrain_model_path:
    print('Downloaded the model in:', pretrain_model_path)
else:
    print('Failed to download the pretrained model.')

Set the environment variable for pre-trained model for the first model:

'PRETRAINED_MODEL': the path where is the pretrained_model exists and is downloaded.

NOTE: You can change the value of 'PRETRAINED_MODEL' by changing its assignment in the cell below.

In [ ]:
os.environ["PRETRAINED_MODEL"]=pretrain_model_path
print("Pretrain_model_path:", os.environ["PRETRAINED_MODEL"])

### Step 2.4: Set the log directory or output directory

'OUTPUT_DIR': the output directory path where model output logs are collected.

The default directory name for the output logs is \"logs\" in the current working directory. You can change this directory name by replacing the value assigned to log_directory in the cell below

In [ ]:
# Set the log directory/output directory to store the logs
log_directory= config_1.notebook_root + os.sep + "logs"
print(log_directory)

os.environ['OUTPUT_DIR']=log_directory
print("Output directory path is:", os.environ['OUTPUT_DIR'])

### Step 2.5:  Run the first Model 

**if BATCH_SIZE is specified per model specification, set it for optimal batch size**

In [18]:
if selected_model['set-batch-size']['cores']:
    import subprocess

    # Run the shell command to get the number of cores
    output = subprocess.check_output("lscpu | grep 'Core(s) per socket' | awk '{print $4}'", shell=True)

    # Decode the byte string output into a regular Python string
    cores = int(output.decode("utf-8").strip())

    cores = eval(selected_section['set-batch-size']['expr'])

**OPTIONAL: You can change the batch size from the model's default. For online_inference,  set batch_size value to be 1.** 

**Run the next cell if you want to change the batch-size, other skip running the next cell**

In [ ]:
batch_size= input('Set the value for batch size that you want to run: ')
os.environ["BATCH_SIZE"]=batch_size

Run the first model using the quickstart script configured in the ai_reference_models.ini file, and save output logs to the selected log directory.

In [ ]:
import os
import subprocess

# Ensure the AI Reference Root is set and navigate to it
ai_reference_root = os.environ.get('AIReferenceRoot', None)
if not ai_reference_root or not os.path.isdir(ai_reference_root):
    raise FileNotFoundError("AIReferenceRoot environment variable is not set or points to an invalid directory.")
os.chdir(ai_reference_root)

# Get the YAML file name from environment variables
yaml_file = os.environ.get('yaml_file', None)
if not yaml_file:
    raise ValueError("yaml_file environment variable is not set.")

# Debugging: Print the YAML file value
print(f"yaml_file: {yaml_file}")

# Construct the config path
configpath = os.path.join(ai_reference_root, "notebooks", yaml_file)

# Debugging: Print the full path to the YAML file
print(f"Constructed configpath: {configpath}")

# Check if the configuration file exists
if not os.path.isfile(configpath):
    raise FileNotFoundError(f"Configuration file not found: {configpath}")

config_1 = AIReferenceConfigFile(os.environ['yaml_file'], os.getcwd() + '/..')
# Build log name
parts = config_1.model_name.split()
log_name = '-'.join(parts) + ".log"

# Check for hardware and framework combination
if general_config_1.hardware_dropdown.value == 'CPU' and general_config_1.framework_dropdown.value == 'TensorFlow':
    # Build command for TensorFlow CPU workload
    run_workload = f"models_v2/{config_1.framework}/{config_1.model_name}/{config_1.mode}/cpu/{config_1.script}"
    log_name = f"{config_1.model_name}-{config_1.framework}-cpu.log"
    cmd = f"./{run_workload} precision={config_1.precision} | tee {log_directory}/{log_name}"

    print("Running:", cmd)
    subprocess.call(cmd, shell=True)

else:
    # Construct the directory path
    dir_path = os.path.normpath(f"models_v2/{config_1.framework}/{config_1.model_name}/{config_1.mode}/{config_1.device}")

    # Debugging: Print the final directory path
    print(f"Final dir_path: {dir_path}")

    if not os.path.isdir(dir_path):
        raise FileNotFoundError(f"Directory not found: {dir_path}")

    # Navigate to the directory
    os.chdir(dir_path)
    print(f"Running in directory: {dir_path}")

    # Run setup and model scripts
    subprocess.call("bash ./setup.sh", shell=True)
    subprocess.call("bash ./run_model.sh", shell=True)

    # Navigate back to the previous directory
    os.chdir(ai_reference_root)

# Navigate back to the original directory (AI Reference Root)
os.chdir(ai_reference_root)

### Step 2.6 Get the throughput or accuracy of the first model

In [ ]:
# Throughput of first workload:
# Split the model_name into individual parts

import re, statistics
try:
    with open(f'{log_directory}/{log_name}', 'r') as file:
        print("DEBUGGING: FILE DIRECTORY:", f'{log_directory}/{log_name}')
        file_content = file.read()
        throughput_matches = re.findall(r'Throughput[^:]*:\s*(\d+\.\d+)\s+fps', file_content)
        accuracy_matches = re.findall(r'Accuracy[^:]*:\s*(\d+\.\d+)', file_content)
        if throughput_matches:
            throughput_matches = [float(x) for x in throughput_matches]
            throughput_value = float(statistics.mean(throughput_matches))
            print(f'Throughput: {throughput_value} fps')
        else:
            print('Throughput not found in the file.')
        if accuracy_matches:
            accuracy_matches = [float(x) for x in accuracy_matches]
            accuracy_value = float(statistics.mean(accuracy_matches))
            print(f'Accuracy: {accuracy_value}')
        else:
            print('Accuracy not found in the file')



except FileNotFoundError:
    print('File not found.')

## Step 3: Select and run the second model

Let's now run a second model and compare its performance with the first.

### Step 3.1 Select another model for comparision

After the list of models is displayed, select one by entering the model index number. (If the environment variable MODEL_2_INDEX is set, we'll use that instead of prompting for input.)

In [ ]:
%cd $notebook_root

In [ ]:
general_config_2 = GeneralConfigFile(ai_root=os.getcwd()+'/..')
general_config_2.toggle_events()

In [ ]:
if (general_config_2.hardware_dropdown.value and general_config_2.framework_dropdown.value and (general_config_2.hardware_dropdown.value == 'CPU' or (general_config_2.hardware_dropdown.value == 'GPU' and general_config_2.gpu_series_dropdown.value))):
    current_dir = current_directory = os.getcwd()
    selected_hardware = general_config_2.hardware_dropdown.value
    selected_framework = general_config_2.framework_dropdown.value

    if selected_framework == 'PyTorch':
        if selected_hardware == 'CPU':
            !python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
            !python -m pip install intel-extension-for-pytorch
            !python -m pip install oneccl_bind_pt --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/

            print("PyTorch CPU packages installed.")

            !./profiling/pytorch_cpu_setup.sh

            !pip install packaging intel-openmp

            os.environ["LD_PRELOAD"] = f"{current_dir}/jemalloc/lib/libjemalloc.so:{current_dir}/tcmalloc/lib/libtcmalloc.so:{general_config_1.root}/new_env/lib/libiomp5.so"
            os.environ["MALLOC_CONF"] = "oversize_threshold:1,background_thread:true,metadata_thp:auto,dirty_decay_ms:9000000000,muzzy_decay_ms:9000000000"
            os.environ["DNNL_MAX_CPU_ISA"] = "AVX512_CORE_AMX"
            os.environ['yaml_file'] = 'profiling/ai_reference_models_pytorch_cpu.yaml'

            print("Finish building PyTorch for CPU")

        elif selected_hardware == 'GPU':
            install_oneapi = general_config_2.intel_oneapi_dropdown.value
            gpu_series = general_config_2.gpu_series_dropdown.value

            if (install_oneapi):
                print("Intel® oneAPI Base Toolkit is installed")
                print("Installing PyTorch for GPU...")
                !pip install torch==2.1.0a0 torchvision==0.16.a0 torchaudio==2.1.0.a0 intel-extension-for-pytorch==2.1.10+xpu --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/xpu/us/
                print("IPEX GPU packages installed.")

                if gpu_series == 'Flex':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_flex_gpu.yaml"
                elif gpu_series == 'Max':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_max_gpu.yaml"
                elif gpu_series == 'Arc':
                    os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_pytorch_arc_gpu.yaml"

                oneapi_path = general_config_2.oneapi_path_input.value.strip()
                print("Finish building IPEX for GPU")
                os.environ['oneAPI_PATH'] = oneapi_path
                oneAPI_PATH = os.environ.get('oneAPI_PATH', '')
                script_content = f"""
                source /{oneAPI_PATH}/compiler/latest/env/vars.sh
                source /{oneAPI_PATH}/mkl/latest/env/vars.sh
                source /{oneAPI_PATH}/tbb/latest/env/vars.sh
                source /{oneAPI_PATH}/mpi/latest/env/vars.sh
                source /{oneAPI_PATH}/ccl/latest/env/vars.sh
                """
                subprocess.run(["/bin/bash", "-c", script_content], check=True)
                print("One API sourcing done.")
                print("PyTorch GPU packages installed.")

    elif selected_framework == 'TensorFlow':
        if selected_hardware == 'CPU':
            !pip cache purge
            !pip install --upgrade tf-nightly-cpu
            !pip install --upgrade tf-keras-nightly
            !pip install --upgrade tf-estimator-nightly
            print("TensorFlow packages installed.")
            os.environ['yaml_file'] = 'profiling/ai_reference_models_tensorflow_cpu.yaml'
            print("Finish building TF for CPU")

        elif selected_hardware == 'GPU':
            gpu_series = general_config_2.gpu_series_dropdown.value
            print("Installing TensorFlow and ITEX for GPU...")
            !pip install tensorflow==2.15.0

            !pip install --upgrade intel-extension-for-tensorflow[xpu]

            if gpu_series == 'Flex':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_flex_gpu.yaml"
            elif gpu_series == 'Max':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_max_gpu.yaml"
            elif gpu_series == 'Arc':
                os.environ['yaml_file'] = f"{current_dir}/profiling/ai_reference_models_tensorflow_arc_gpu.yaml"
            print("TensorFlow GPU packages installed.")

else:
    raise Exception("Please rerun the selections to ensure correct installations.")

# Run the second model
This notebook helps you compare the performance of two models listed in the supported models. Select the first model to run and compare. (If the environment variable MODEL_1_INDEX is set, we'll use that instead of prompting for input.)


In [ ]:
config_2 = AIReferenceConfigFile(os.environ['yaml_file'], os.getcwd() + '/..')
notebook_root = config_2.notebook_root
%cd $notebook_root
selected_model = config_2.model_selection()
print(selected_model)
selected_model = config_2.device_specific(selected_model, general_config_2.hardware_dropdown.value, os.environ['yaml_file'], selected_framework)
os.environ['PRECISION'] = config_2.precision
if selected_framework == 'TensorFlow' and selected_hardware == 'CPU':
    selected_script = config_2.script
else:
    os.environ['TEST_MODE'] = config_2.test_mode

### Additional Setup
The following cell checks for additional environment variables that needs to be specified per model specification. 

Then checks for additional files, libraries, and configurations that needs to be installed per model specification. Commands are installed in a chronological dependency order.

In [ ]:
import subprocess
ai_reference_root = config_2.ai_root
%cd $ai_reference_root

for export in selected_model['exports']:
    env_name = export.split(' ')[0]
    env_value = export.split(' ')[1]
    if env_value == '-i':
        env_value = input(f'Please provide the path for the following environment variable {env_name}: ', )
    os.environ[env_name] = os.path.expandvars(env_value)

    print("Assigned environment variable ", env_name, " : ", os.environ[env_name])

for cmd in selected_model['additional-commands']:
    if 'os.environ' in cmd: exec(cmd)
    else : subprocess.call(cmd, shell=True)

In [ ]:
import os
import subprocess

# Retrieve the AI reference root from environment variables
ai_reference_root = os.environ.get('AIReferenceRoot')
if not ai_reference_root:
    raise ValueError("Environment variable 'AIReferenceRoot' is not set.")

# Change to the specified directory
os.chdir(ai_reference_root)

# Process exports from the selected model
exports = selected_model.get('exports', [])
for export in exports:
    try:
        # Split the export string into name and value
        env_parts = export.split(' ', 1)
        if len(env_parts) != 2:
            raise ValueError(f"Invalid export format: {export}")

        env_name, env_value = env_parts
        if env_value == '-i':
            # Prompt for user input if value is '-i'
            env_value = input(f"Please provide the path for the environment variable '{env_name}': ")

        # Expand and set the environment variable
        os.environ[env_name] = os.path.expandvars(env_value)
        print(f"Assigned environment variable {env_name}: {os.environ[env_name]}")

    except Exception as e:
        print(f"Error processing export '{export}': {e}")

# Process additional commands from the selected model
additional_commands = selected_model.get('additional-commands', [])
for cmd in additional_commands:
    try:
        if 'os.environ' in cmd:
            # Execute Python commands directly
            exec(cmd)
        else:
            # Execute shell commands
            subprocess.call(cmd, shell=True)
        print(f"Executed command: {cmd}")

    except Exception as e:
        print(f"Error executing command '{cmd}': {e}")


### Step 3.2: Get the required dataset for the selected model

The following cell checks for the second dataset path. If the data-location is already specified, then the notebook will use the dataset path mentioned in ai_reference_models.ini file. If the data-location is not specified in the ai_reference_models.ini, the data downloading option instructions are shown. You must manually download the dataset using these instructions before you can proceed to the next cell.

Set the path for dataset directory for the second model:

DATASET_DIR': the path where the dataset exists and is downloaded.

**ACTION : You need to input the path where the dataset for the second model exists or where you have downloaded it in your system**

In [ ]:
dataset_path = input('Input the path where the dataset exists for the first model:')

os.environ['DATASET_DIR'] = dataset_path

print("Data location path:", os.environ['DATASET_DIR'])

### Step 3.3: Prepare pre-trained model for the selected model

This step checks if the pre-trained model for the selected model exists in the pre-trained directory path. If the pre-trained directory does not exist, then it downloads the pre-trained model for the selected precision.

In [ ]:
wget_value = config_2.wget

pretrain_model_path = config_2.download_pretrained_model(wget=wget_value, current_path=config_2.notebook_root)
pretrain_model_path = config_2.uncompress_file(pretrain_model_path, current_path=config_2.notebook_root)
if pretrain_model_path:
    print('Downloaded the model in:', pretrain_model_path)
else:
    print('Failed to download the pretrained model.')

Set the environment variable for pre-trained model for the second model:

'PRETRAINED_MODEL': the path where is the pretrained_model exists and is downloaded.

NOTE: You can change the value of 'PRETRAINED_MODEL' by changing its assignment in the cell below.

In [ ]:
os.environ["PRETRAINED_MODEL"]=pretrain_model_path
print("Pretrain_model_path:", os.environ["PRETRAINED_MODEL"])

### Step 3.4: Set the log directory or output directory

'OUTPUT_DIR': the output directory path where model output logs are collected.

The default directory name for the output logs is "logs" in the current working directory. You can change this directory name by replacing the value assigned to log_directory in the cell below

In [ ]:
log_directory= config_2.notebook_root + os.sep + "logs"
print(log_directory)

os.environ['OUTPUT_DIR']=log_directory
print("Output directory path is:", os.environ['OUTPUT_DIR'])

### Step 3.5: Run the second model

**if BATCH_SIZE is specified per model specification, set it for optimal batch size**

In [ ]:
if selected_model['set-batch-size']['cores']:
    import subprocess

    # Run the shell command to get the number of cores
    output = subprocess.check_output("lscpu | grep 'Core(s) per socket' | awk '{print $4}'", shell=True)

    # Decode the byte string output into a regular Python string
    cores = int(output.decode("utf-8").strip())

    cores = eval(selected_section['set-batch-size']['expr'])

**OPTIONAL: You can change the batch size from the model's default. For online_inference,  set batch_size value to be 1.** 

**Run the next cell if you want to change the batch-size, other skip running the next cell**

In [ ]:
batch_size= input('Set the value for batch size that you want to run: ')
os.environ["BATCH_SIZE"]=batch_size

Run the second model using the quickstart script configured in the ai_reference_models.ini file, and save output logs to the selected log directory.

In [ ]:
import os
import subprocess

# Ensure the AI Reference Root is set and navigate to it
ai_reference_root = os.environ.get('AIReferenceRoot', None)
if not ai_reference_root or not os.path.isdir(ai_reference_root):
    raise FileNotFoundError("AIReferenceRoot environment variable is not set or points to an invalid directory.")
os.chdir(ai_reference_root)

# Get the YAML file name from environment variables
yaml_file = os.environ.get('yaml_file', None)
if not yaml_file:
    raise ValueError("yaml_file environment variable is not set.")

# Debugging: Print the YAML file value
print(f"yaml_file: {yaml_file}")

# Construct the config path
configpath = os.path.join(ai_reference_root, "notebooks", yaml_file)

# Debugging: Print the full path to the YAML file
print(f"Constructed configpath: {configpath}")

# Check if the configuration file exists
if not os.path.isfile(configpath):
    raise FileNotFoundError(f"Configuration file not found: {configpath}")

config_2 = AIReferenceConfigFile(os.environ['yaml_file'], os.getcwd() + '/..')
# Build log name
parts = config_2.model_name.split()
log_name = '-'.join(parts) + ".log"

# Check for hardware and framework combination
if general_config_2.hardware_dropdown.value == 'CPU' and general_config_2.framework_dropdown.value == 'TensorFlow':
    # Build command for TensorFlow CPU workload
    run_workload = f"models_v2/{config_2.framework}/{config_2.model_name}/{config_2.mode}/cpu/{config_2.script}"
    log_name = f"{config_2.model_name}-{config_2.framework}-cpu.log"
    cmd = f"./{run_workload} precision={config_2.precision} | tee {log_directory}/{log_name}"

    print("Running:", cmd)
    subprocess.call(cmd, shell=True)

else:
    # Construct the directory path
    dir_path = os.path.normpath(f"models_v2/{config_2.framework}/{config_2.model_name}/{config_2.mode}/{config_2.device}")

    # Debugging: Print the final directory path
    print(f"Final dir_path: {dir_path}")

    if not os.path.isdir(dir_path):
        raise FileNotFoundError(f"Directory not found: {dir_path}")

    # Navigate to the directory
    os.chdir(dir_path)
    print(f"Running in directory: {dir_path}")

    # Run setup and model scripts
    subprocess.call("bash ./setup.sh", shell=True)
    subprocess.call("bash ./run_model.sh", shell=True)

    # Navigate back to the previous directory
    os.chdir(ai_reference_root)

# Navigate back to the original directory (AI Reference Root)
os.chdir(ai_reference_root)

### Step 3.5 Get the throughput or accuracy of the second model

In [ ]:

# Throughput of second workload:
# Split the model_name into individual parts
import re, statistics
try:
    with open(f'{log_directory}/{log_name}', 'r') as file:
        print("DEBUGGING: FILE DIRECTORY:", f'{log_directory}/{log_name}')
        file_content = file.read()
        throughput_matches = re.findall(r'Throughput[^:]*:\s*(\d+\.\d+)\s+fps', file_content)
        accuracy_matches = re.findall(r'Accuracy[^:]*:\s*(\d+\.\d+)', file_content)
        if throughput_matches:
            throughput_matches = [float(x) for x in throughput_matches]
            throughput_value = float(statistics.mean(throughput_matches))
            print(f'Throughput: {throughput_value} fps')
        else:
            print('Throughput not found in the file.')
        if accuracy_matches:
            accuracy_matches = [float(x) for x in accuracy_matches]
            accuracy_value = float(statistics.mean(accuracy_matches))
            print(f'Accuracy: {accuracy_value}')
        else:
            print('Accuracy not found in the file')

## Step 4: Compare performance results and plot a comparison chart for the two models

Get the results (throughput/accuracy) of the two models for comparision

In [ ]:
print("Throughput of", model_1_name, ": ", Throughput_of_workload_1)
print("Throughput of", model_2_name, ": ", Throughput_of_workload_2)

Plot a chart for comparision 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
fig, ax = plt.subplots()

# Define your data
categories = [model_1_name.split(' ')[0] + selected_precision, model_2_name.split(' ')[0] + selected_precision_2]

# float array expected for charts.
values = [float(Throughput_of_workload_1), float(Throughput_of_workload_2)]


# Generate a list of colors for each bar
colors = ['blue', 'orange']


print(categories, values, colors)

ax.bar(x=categories, height=values, color=colors)
ax.set_ylabel("Throughput")
ax.set_title("Throughput by each model")
plt.show()